In [1]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!wget https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv -O titanic.csv

--2025-06-10 17:31:11--  https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv
Resolving web.stanford.edu (web.stanford.edu)... 171.67.215.200, 2607:f6d0:0:925a::ab43:d7c8
Connecting to web.stanford.edu (web.stanford.edu)|171.67.215.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44225 (43K) [text/csv]
Saving to: ‘titanic.csv’

100%[======================================>] 44,225      --.-K/s   in 0.1s    

2025-06-10 17:31:12 (416 KB/s) - ‘titanic.csv’ saved [44225/44225]



In [9]:
import pandas as pd

df = pd.read_csv("data/Corporate_Actions.csv")
print(df.shape)
print(df.columns.tolist())

(11095, 11)
['Security Code', 'Security Name', 'Company Name', 'Ex Date', 'Purpose', 'Record Date', 'BC Start Date', ' BC End Date\t', 'ND Start Date', 'ND End Date', 'Actual Payment Date']


In [ ]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///corporate.db")
df.to_sql("corporate_action", engine, index=False)

In [ ]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM corporate_action ;"))

In [19]:
# Ensure your AWS credentials are configured

from langchain.chat_models import init_chat_model

llm = init_chat_model("us.anthropic.claude-3-5-sonnet-20240620-v1:0", model_provider="bedrock_converse")

In [20]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [21]:
agent_executor.invoke({"input": "what's Dividend paid by HCLtech"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded: [{'type': 'tool_use', 'name': 'sql_db_list_tables', 'id': 'tooluse_RE-IgItFQsa4gO3hSQ393w', 'index': 0, 'input': '{"tool_input": ""}'}]

corporate, corporate_action
Invoking: `sql_db_schema` with `{'table_names': 'corporate, corporate_action'}`
responded: [{'type': 'text', 'text': "\n\nIt seems the most relevant tables for this query are 'corporate' and 'corporate_action'. Let's check their schemas to see which one contains dividend information.", 'index': 0}, {'type': 'tool_use', 'name': 'sql_db_schema', 'id': 'tooluse_ZgK-r9ZFTMWZEkULbbA-BQ', 'index': 1, 'input': '{"table_names": "corporate, corporate_action"}'}]


CREATE TABLE corporate (
	"Security Code" BIGINT, 
	"Security Name" TEXT, 
	"Company Name" TEXT, 
	"Ex Date" TEXT, 
	"Purpose" TEXT, 
	"Record Date" TEXT, 
	"BC Start Date" TEXT, 
	" BC End Date	" TEXT, 
	"ND Start Date" TEXT, 
	"ND End Date" TEXT, 
	"Actual Pa

{'input': "what's Dividend paid by HCLtech",
 'output': [{'type': 'text',
   'text': '\n\n\n\n\n\nBased on the query results, I can provide you with information about the dividends paid by HCL Technologies Ltd (HCLtech):\n\n1. The most recent dividend payment was an Interim Dividend of Rs. 18.0000 per share, with an Ex-Date of 28 Apr 2023.\n\n2. In the previous year (2022), they also paid an Interim Dividend of Rs. 18.0000 per share, with an Ex-Date of 28 Apr 2022.\n\n3. In 2021, they paid multiple dividends:\n   - Interim Dividend of Rs. 6.0000 and a Special Dividend of Rs. 10.0000, both with an Ex-Date of 29 Apr 2021.\n   - Another Interim Dividend of Rs. 6.0000 with an Ex-Date of 27 Jul 2021.\n\n4. In 2020, they paid two Interim Dividends of Rs. 2.0000 each, with Ex-Dates of 24 Jan 2020 and 23 Jul 2020.\n\n5. There are also future dividend payments listed in the database:\n   - Interim Dividend of Rs. 18.0000 scheduled for 28 Apr 2025.\n   - Two Interim Dividends of Rs. 12.0000 each

In [16]:
agent_executor.invoke({"input": "Curently I am in the year 2025.what is bonus issue by BPCL security"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded: [{'type': 'text', 'text': '  Only after that I should formulate the actual query to answer the question.\n\nTo start, let me list the tables in the database:', 'index': 0}, {'type': 'tool_use', 'name': 'sql_db_list_tables', 'id': 'tooluse_l5BCW_M3QIOdr0ZYUBb6VQ', 'index': 1, 'input': '{"tool_input": ""}'}]

corporate, corporate_action
Invoking: `sql_db_schema` with `{'table_names': 'corporate_action'}`
responded: [{'type': 'text', 'text': "\n\nBased on the table names, it looks like the 'corporate_action' table might contain the information we need about bonus issues. Let's check its schema:", 'index': 0}, {'type': 'tool_use', 'name': 'sql_db_schema', 'id': 'tooluse_nXSCEsu9RWGcRViLOXGl-A', 'index': 1, 'input': '{"table_names": "corporate_action"}'}]


CREATE TABLE corporate_action (
	"Security Code" BIGINT, 
	"Security Name" TEXT, 
	"Company Name" TEXT, 
	"Ex Date" TEXT, 


{'input': 'Curently I am in the year 2025.what is bonus issue by BPCL security',
 'output': [{'type': 'text',
   'text': "\n\n\n\nBased on the information from our database, I can provide the following answer:\n\nCurrently, in the year 2025, there is no record of a bonus issue by BPCL (Bharat Petroleum Corporation Ltd) security in our database. \n\nThe most recent corporate actions for BPCL in our database are:\n\n1. An interim dividend of Rs. 21.0000 with an ex-date of 12 Dec 2023\n2. A dividend of Rs. 4.0000 with an ex-date of 11 Aug 2023\n3. A final dividend of Rs. 6.0000 with an ex-date of 19 Aug 2022\n\nOur database contains corporate actions for BPCL up to the end of 2023, but there are no records of any bonus issues within this timeframe. It's possible that:\n\n1. BPCL hasn't announced any bonus issues for 2025 yet.\n2. If such an announcement has been made, it hasn't been recorded in our database.\n3. BPCL might not have plans for a bonus issue in 2025.\n\nFor the most up-to-da

In [13]:
agent_executor.invoke({"input": "what's is your name "})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded: [{'type': 'text', 'text': "  Finally, I should query the database to answer the question.\n\nI don't know the names of the tables in the database, so I'll start by listing them:", 'index': 0}, {'type': 'tool_use', 'name': 'sql_db_list_tables', 'id': 'tooluse_v3w-oGnNRzKjvj20Ao-GKQ', 'index': 1, 'input': '{"tool_input": ""}'}]

titanic[{'type': 'text', 'text': '\n\nThe database contains only one table: "titanic". This table is unlikely to contain information about my name. As an AI assistant, I don\'t have a personal name, and the information about me wouldn\'t be stored in a database about the Titanic.\n\nGiven this information, I can confidently say:\n\nI don\'t know. The database available to me contains information about the Titanic, which is not related to my name or identity as an AI assistant.', 'index': 0}]

> Finished chain.


{'input': "what's is your name ",
 'output': [{'type': 'text',
   'text': '\n\nThe database contains only one table: "titanic". This table is unlikely to contain information about my name. As an AI assistant, I don\'t have a personal name, and the information about me wouldn\'t be stored in a database about the Titanic.\n\nGiven this information, I can confidently say:\n\nI don\'t know. The database available to me contains information about the Titanic, which is not related to my name or identity as an AI assistant.',
   'index': 0}]}